# Packages

In [1]:
import pandas as pd
import numpy as np
import os
import folium

import keras

import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.ticker as ticker
from matplotlib.ticker import PercentFormatter

pd.options.display.max_columns = None
pd.options.display.max_rows = None

Using TensorFlow backend.


In [2]:
from lib_Map import plot_map
from lib_Dummies import Master_dummies
from lib_NNT import NNTRegressor
from lib_Tiempo import tiempo
from lib_Temperatura import temperatura
from lib_Presion import Presion
from lib_Viento import Viento
from lib_Densidad import densidad

# Load data

In [3]:
Data_train = pd.read_excel("./csv/Modelo de Consumo Predictivo.xlsx", sheet_name = "Data")
print(Data_train.shape)
Data_test = pd.read_excel("./csv/Modelo de Consumo Predictivo.xlsx", sheet_name = "Eval")
print(Data_test.shape)

(440806, 10)
(148931, 9)


In [4]:
Data_train.columns

Index(['ID', 'ARTICULO', 'FECHA', 'CENTROCONSUMO', 'UBICACIONVIRTUAL',
       'ID_ORGANOGESTOR', 'DESC_PROVINCIA', 'FECHACADUCIDADEXISTENCIAS',
       'IMPORTESALIDA', 'CANTIDADSALIDA'],
      dtype='object')

In [5]:
Data_test.columns

Index(['ID', 'ARTICULO', 'FECHA', 'CENTROCONSUMO', 'UBICACIONVIRTUAL',
       'ID_ORGANOGESTOR', 'DESC_PROVINCIA', 'FECHACADUCIDADEXISTENCIAS',
       'IMPORTESALIDA'],
      dtype='object')

# Data Engineer

## Creación variables año, mes y día:

### Train:

In [6]:
Data_train['Año'] = [i.year for i in Data_train['FECHA']]
Data_train['Mes'] = [i.month for i in Data_train['FECHA']]
Data_train['Dia'] = [i.day for i in Data_train['FECHA']]

### Train:

In [7]:
Data_test['Año'] = [i.year for i in Data_test['FECHA']]
Data_test['Mes'] = [i.month for i in Data_test['FECHA']]
Data_test['Dia'] = [i.day for i in Data_test['FECHA']]

In [8]:
Data_train['Año'].unique().tolist()

[2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015]

## Conversión a datos mensuales:

In [9]:
del Data_train['ID'], Data_train['FECHA'], Data_train['FECHACADUCIDADEXISTENCIAS'], Data_train['Dia']

In [10]:
Data_train.head()

,ARTICULO,CENTROCONSUMO,UBICACIONVIRTUAL,ID_ORGANOGESTOR,DESC_PROVINCIA,IMPORTESALIDA,CANTIDADSALIDA,Año,Mes
0,-1073438649,17414,1525,3920000000000360380,Córdoba,0.5292,12.0,2008,1
1,-1073438649,17414,1525,3920000000000360380,Córdoba,1.3230,30.0,2008,1
2,-1073438649,17416,1527,3920000000000360380,Córdoba,-0.1323,-3.0,2008,1
3,-1073438649,17416,1527,3920000000000360380,Córdoba,-0.1323,-3.0,2008,1
4,-1073438649,17414,1525,3920000000000360380,Córdoba,0.1323,3.0,2008,1


### Train:

In [11]:
Data_train_mensual = (Data_train.groupby(('ARTICULO', 'CENTROCONSUMO', 'DESC_PROVINCIA',  'Año', 'Mes')) # Agrupar
          .CANTIDADSALIDA    # Quedarse con la columna CANTIDADSALIDA
          .apply(sum)       # Calcular su suma
          .reset_index())   # Deshacer índice jerárquico

C:\Users\victo\Anaconda3\envs\baseFSA\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Interpreting tuple 'by' as a list of keys, rather than a single key. Use 'by=[...]' instead of 'by=(...)'. In the future, a tuple will always mean a single key.
  """Entry point for launching an IPython kernel.


In [12]:
Data_train_mensual.head()

,ARTICULO,CENTROCONSUMO,DESC_PROVINCIA,Año,Mes,CANTIDADSALIDA
0,-1073449806,1021,Granada,2012,3,2.0
1,-1073449806,9517,Granada,2012,1,2.0
2,-1073449806,9517,Granada,2012,4,2.0
3,-1073449806,9517,Granada,2012,5,1.0
4,-1073449806,9538,Granada,2012,1,-20.0


### Test

In [13]:
Data_test_mensual =  Data_test[['ARTICULO', 'CENTROCONSUMO', 'DESC_PROVINCIA', 'Año', 'Mes']]
Data_test_mensual.drop_duplicates(keep='first',inplace=True) 

C:\Users\victo\Anaconda3\envs\baseFSA\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


## Deficiencias

In [14]:
len(Data_train_mensual['ARTICULO'].unique().tolist())

20

In [15]:
len(Data_test_mensual['ARTICULO'].unique().tolist())

11

In [16]:
l=[]
for i in Data_test_mensual['ARTICULO'].unique().tolist():
    if i not in Data_train_mensual['ARTICULO'].unique().tolist():
        l.append(i)
len(l)

1

## Sort values and saving as csv

In [17]:
Data_train_mensual = Data_train_mensual.sort_values(['Año', 'Mes', 'CENTROCONSUMO' ])
print(Data_train_mensual.shape)
Data_train_mensual.to_csv('./csv/clean.csv')

(67688, 6)


In [18]:
Data_train_mensual.head()

,ARTICULO,CENTROCONSUMO,DESC_PROVINCIA,Año,Mes,CANTIDADSALIDA
3410,-1073438649,17405,Córdoba,2008,1,8.0
3416,-1073438649,17407,Córdoba,2008,1,40.0
3421,-1073438649,17409,Córdoba,2008,1,40.0
3430,-1073438649,17414,Córdoba,2008,1,293.0
3471,-1073438649,17415,Córdoba,2008,1,59.0


In [19]:
Data_train_mensual.head()

,ARTICULO,CENTROCONSUMO,DESC_PROVINCIA,Año,Mes,CANTIDADSALIDA
3410,-1073438649,17405,Córdoba,2008,1,8.0
3416,-1073438649,17407,Córdoba,2008,1,40.0
3421,-1073438649,17409,Córdoba,2008,1,40.0
3430,-1073438649,17414,Córdoba,2008,1,293.0
3471,-1073438649,17415,Córdoba,2008,1,59.0


## Nuevas variables

### Horas de Sol y precipitaciones:

In [20]:
# Identfy the file location of the master dataset
I_data_path = './csv/clean.csv' 

# Define file name of output .csv file
I_out_filename = 'clean_lluvia'
I_data_tiempo = './csv/lluvia/'

pd_tiempo, Data_train_lluvia = tiempo(I_data_path, I_out_filename, I_data_tiempo)

(67688, 6)
Saving dataset to ./csv/clean_lluvia.csv


In [21]:
pd_tiempo.shape

(960, 6)

In [22]:
Data_train_lluvia.head()

,ARTICULO,CENTROCONSUMO,DESC_PROVINCIA,Año,Mes,CANTIDADSALIDA,Horas_Sol,Precipitacion
0,-1073438649,17405,Córdoba,2008,1,8.0,171.5,70.0
1,-1073438649,17407,Córdoba,2008,1,40.0,171.5,70.0
2,-1073438649,17409,Córdoba,2008,1,40.0,171.5,70.0
3,-1073438649,17414,Córdoba,2008,1,293.0,171.5,70.0
4,-1073438649,17415,Córdoba,2008,1,59.0,171.5,70.0


### Temperatura

In [23]:
# Identfy the file location of the master dataset
I_data_path = './csv/clean_lluvia.csv' 

# Define file name of output .csv file
I_out_filename = 'clean_temperatura'
I_data_tiempo = './csv/Temperatura/'

pd_temp, Data_train_temp = temperatura(I_data_path, I_out_filename, I_data_tiempo)

(67688, 8)
Saving dataset to ./csv/clean_temperatura.csv


In [24]:
pd_temp

,Provincia,Año,Mes,Temp_Max,Key_merge,Temp_Min
0,Almería,2008,1,17.570968,Almería20081,9.225806
1,Almería,2008,2,17.975862,Almería20082,11.241379
2,Almería,2008,3,19.587097,Almería20083,11.132258
3,Almería,2008,4,21.783333,Almería20084,13.463333
4,Almería,2008,5,22.248387,Almería20085,15.387097
5,Almería,2008,6,27.146667,Almería20086,19.313333
6,Almería,2008,7,29.654839,Almería20087,22.670968
7,Almería,2008,8,31.041935,Almería20088,23.567742
8,Almería,2008,9,27.383333,Almería20089,20.516667
9,Almería,2008,10,24.164516,Almería200810,17.103226


In [25]:
Data_train_temp.tail()

,ARTICULO,CENTROCONSUMO,DESC_PROVINCIA,Año,Mes,CANTIDADSALIDA,Horas_Sol,Precipitacion,Temp_Max,Temp_Min
67683,2147471022,365878,Málaga,2015,12,4.0,148.9,0.8,20.461290,11.074194
67684,2147470713,369578,Cádiz,2015,12,10.0,164.5,33.4,19.287097,13.119355
67685,2147470713,372677,Cádiz,2015,12,40.0,164.5,33.4,19.287097,13.119355
67686,2147470713,372977,Sevilla,2015,12,2500.0,199.7,18.6,20.174194,8.300000
67687,2147470713,373592,Cádiz,2015,12,50.0,164.5,33.4,19.287097,13.119355


### Presión:

In [26]:
# Identfy the file location of the master dataset
I_data_path = './csv/clean_temperatura.csv' 

# Define file name of output .csv file
I_out_filename = 'clean_presion'

I_data_Presion = './csv/Presion/'

pd_Presion, Data_train_Presion = Presion(I_data_path, I_out_filename, I_data_Presion)

(67688, 10)
Saving dataset to ./csv/clean_presion.csv


In [27]:
Data_train_Presion.head()

,ARTICULO,CENTROCONSUMO,DESC_PROVINCIA,Año,Mes,CANTIDADSALIDA,Horas_Sol,Precipitacion,Temp_Max,Temp_Min,Presion_Max,Presion_Min
0,-1073438649,17405,Córdoba,2008,1,8.0,171.5,70.0,16.93871,5.119355,1016.954839,1011.587097
1,-1073438649,17407,Córdoba,2008,1,40.0,171.5,70.0,16.93871,5.119355,1016.954839,1011.587097
2,-1073438649,17409,Córdoba,2008,1,40.0,171.5,70.0,16.93871,5.119355,1016.954839,1011.587097
3,-1073438649,17414,Córdoba,2008,1,293.0,171.5,70.0,16.93871,5.119355,1016.954839,1011.587097
4,-1073438649,17415,Córdoba,2008,1,59.0,171.5,70.0,16.93871,5.119355,1016.954839,1011.587097


In [28]:
pd_Presion.head()

,Provincia,Año,Mes,Presion_Max,Key_merge,Presion_Min
0,Almería,2008,1,1024.212903,Almería20081,1019.106452
1,Almería,2008,2,1023.762069,Almería20082,1019.468966
2,Almería,2008,3,1016.051613,Almería20083,1010.316129
3,Almería,2008,4,1014.856667,Almería20084,1009.240000
4,Almería,2008,5,1011.254839,Almería20085,1007.219355


### Viento:

In [29]:
# Identfy the file location of the master dataset
I_data_path = './csv/clean_presion.csv' 

# Define file name of output .csv file
I_out_filename = 'clean_viento'

I_data_viento = './csv/Viento/'

pd_viento, Data_train_Viento = Viento(I_data_path, I_out_filename, I_data_viento)

(67688, 12)
Saving dataset to ./csv/clean_viento.csv


In [30]:
pd_viento.head()

,Provincia,Año,Mes,Viento_Max,Key_merge,Viento_Min
0,Almería,2008,1,40.796129,Almería20081,15.143226
1,Almería,2008,2,44.106207,Almería20082,18.496552
2,Almería,2008,3,44.442581,Almería20083,15.642581
3,Almería,2008,4,50.316000,Almería20084,20.700000
4,Almería,2008,5,45.789677,Almería20085,17.941935


In [31]:
Data_train_Viento.head()

,ARTICULO,CENTROCONSUMO,DESC_PROVINCIA,Año,Mes,CANTIDADSALIDA,Horas_Sol,Precipitacion,Temp_Max,Temp_Min,Presion_Max,Presion_Min,Viento_Max,Viento_Min
0,-1073438649,17405,Córdoba,2008,1,8.0,171.5,70.0,16.93871,5.119355,1016.954839,1011.587097,23.469677,5.098065
1,-1073438649,17407,Córdoba,2008,1,40.0,171.5,70.0,16.93871,5.119355,1016.954839,1011.587097,23.469677,5.098065
2,-1073438649,17409,Córdoba,2008,1,40.0,171.5,70.0,16.93871,5.119355,1016.954839,1011.587097,23.469677,5.098065
3,-1073438649,17414,Córdoba,2008,1,293.0,171.5,70.0,16.93871,5.119355,1016.954839,1011.587097,23.469677,5.098065
4,-1073438649,17415,Córdoba,2008,1,59.0,171.5,70.0,16.93871,5.119355,1016.954839,1011.587097,23.469677,5.098065


### CENTROCONSUMO_str

In [32]:
Data = Data_train_Viento.copy()
Data['CENTROCONSUMO'] = Data['CENTROCONSUMO'].astype(str)
Data['CENTROCONSUMO_str'] = [i[0:3] for i in Data['CENTROCONSUMO']]
Data.to_csv('./csv/CCStr.csv' )

### Densidad

In [33]:
# Identfy the file location of the master dataset
I_data_path = './csv/CCStr.csv' 

# Define file name of output .csv file
I_out_filename = 'clean_master_Densidad'

I_data_densidad = './csv/Densidad_poblacion/'

Densidad_df, Data_train_Densidad = densidad(I_data_path, I_out_filename, I_data_densidad)

(67688, 15)
Saving dataset to ./csv/clean_master_Densidad.csv


In [34]:
Data_train_Densidad.head()

,ARTICULO,CENTROCONSUMO,DESC_PROVINCIA,Año,Mes,CANTIDADSALIDA,Horas_Sol,Precipitacion,Temp_Max,Temp_Min,Presion_Max,Presion_Min,Viento_Max,Viento_Min,CENTROCONSUMO_str,Densidad: 0-4 años,Densidad: 5-18 años,Densidad: 19-30 años,Densidad: 31-45 años,Densidad: 46-65 años,Densidad: 66-75 años,Densidad: 76> años
0,-1073438649,17405,Córdoba,2008,1,8.0,171.5,70.0,16.93871,5.119355,1016.954839,1011.587097,23.469677,5.098065,174,42073,123677,134617,185557,178212,66689,61856
1,-1073438649,17407,Córdoba,2008,1,40.0,171.5,70.0,16.93871,5.119355,1016.954839,1011.587097,23.469677,5.098065,174,42073,123677,134617,185557,178212,66689,61856
2,-1073438649,17409,Córdoba,2008,1,40.0,171.5,70.0,16.93871,5.119355,1016.954839,1011.587097,23.469677,5.098065,174,42073,123677,134617,185557,178212,66689,61856
3,-1073438649,17414,Córdoba,2008,1,293.0,171.5,70.0,16.93871,5.119355,1016.954839,1011.587097,23.469677,5.098065,174,42073,123677,134617,185557,178212,66689,61856
4,-1073438649,17415,Córdoba,2008,1,59.0,171.5,70.0,16.93871,5.119355,1016.954839,1011.587097,23.469677,5.098065,174,42073,123677,134617,185557,178212,66689,61856


### Save as csv:

In [35]:
Master = Data_train_Densidad.copy()
Master.to_csv('./csv/Master.csv')

# Limpieza

In [36]:
Master.isnull().sum()

ARTICULO                  0
CENTROCONSUMO             0
DESC_PROVINCIA            0
Año                       0
Mes                       0
CANTIDADSALIDA            0
Horas_Sol                 0
Precipitacion             0
Temp_Max                177
Temp_Min                177
Presion_Max               0
Presion_Min               0
Viento_Max                0
Viento_Min                0
CENTROCONSUMO_str         0
Densidad: 0-4 años        0
Densidad: 5-18 años       0
Densidad: 19-30 años      0
Densidad: 31-45 años      0
Densidad: 46-65 años      0
Densidad: 66-75 años      0
Densidad: 76> años        0
dtype: int64

## Imputación de valores perdidos

### Temp_Max y Temp_Min:

In [37]:
index_empty = Master.loc[Master['Temp_Max'].isnull()].index.tolist()
for i in index_empty:
    A = (Master["CENTROCONSUMO"] == Master["CENTROCONSUMO"][i])
    B = (Master["Mes"] == Master["Mes"][i])
    average_Temp_Max = Master.loc[A & B]['Temp_Max'].mean()
    average_Temp_Max_Tot = Master['Temp_Max'].mean()
    average_Temp_Min = Master.loc[A & B]['Temp_Min'].mean()
    average_Temp_Min_Tot = Master['Temp_Max'].mean()
    if np.isnan(average_Temp_Max):
        Master['Temp_Max'][i] = average_Temp_Max_Tot
        Master['Temp_Min'][i] = average_Temp_Min_Tot
    else:
        Master['Temp_Max'][i] = average_Temp_Max
        Master['Temp_Min'][i] = average_Temp_Min

In [38]:
Master.isnull().sum()

ARTICULO                0
CENTROCONSUMO           0
DESC_PROVINCIA          0
Año                     0
Mes                     0
CANTIDADSALIDA          0
Horas_Sol               0
Precipitacion           0
Temp_Max                0
Temp_Min                0
Presion_Max             0
Presion_Min             0
Viento_Max              0
Viento_Min              0
CENTROCONSUMO_str       0
Densidad: 0-4 años      0
Densidad: 5-18 años     0
Densidad: 19-30 años    0
Densidad: 31-45 años    0
Densidad: 46-65 años    0
Densidad: 66-75 años    0
Densidad: 76> años      0
dtype: int64

## Save as csv

In [39]:
Master.to_csv('./csv/Master_clean.csv')

# Análisis exploratorio

## Mapas:

In [40]:
coord_path = "./csv/listado-longitud-latitud-municipios-espana.xls"
TypeData = Data_train

### Total:

In [41]:
mapas_total = plot_map(coord_path, TypeData, Año = 'Total')

In [42]:
mapas_total[1]

### Por años:

In [43]:
mapas_años = []
for Año in Data_train['Año'].unique().tolist():
    mapas_años.append(plot_map(coord_path, TypeData, Año))

In [44]:
mapas_años[6][0]

# One hot encoding

In [45]:
# Identfy the file location of the master dataset
I_data_path = './csv/Master_clean.csv' 

# Define file name of output .csv file
I_out_filename = 'clean_master_dummies_Big'

# Max number of dummies for each feature allowed (if ones have less that Max, those features will be removed)
Max = float('inf') #to delete this bound

# Function
pd_Master_dummies1 = Master_dummies(I_data_path, I_out_filename, Max)

Dummies features: 
  - ARTICULO: 20 new columns
  - CENTROCONSUMO: 2445 new columns
  - DESC_PROVINCIA: 8 new columns
  - Mes: 12 new columns
Adding dummies to dataset...
Completed
Working dataset has: 67688 rows and 2503 columns
Saving dataset to ./csv/clean_master_dummies_Big.csv


In [46]:
# Identfy the file location of the master dataset
I_data_path = './csv/Master_clean.csv' 

# Define file name of output .csv file
I_out_filename = 'clean_master_dummies_Medium'

# Max number of dummies for each feature allowed (if ones have less that Max, those features will be removed)
Max = 2000  

# Function
pd_Master_dummies2 = Master_dummies(I_data_path, I_out_filename, Max)

Dummies features: 
  - ARTICULO: 20 new columns
  - CENTROCONSUMO: 2445 new columns
    --> CENTROCONSUMO has been removed
  - Mes: 12 new columns
Adding dummies to dataset...
Completed
Working dataset has: 67688 rows and 58 columns
Saving dataset to ./csv/clean_master_dummies_Medium.csv


In [47]:
# Identfy the file location of the master dataset
I_data_path = './csv/Master_clean.csv' 

# Define file name of output .csv file
I_out_filename = 'clean_master_dummies_shorter'

# Max number of dummies for each feature allowed (if ones have less that Max, those features will be removed)
Max = 50  

# Function
pd_Master_dummies3 = Master_dummies(I_data_path, I_out_filename, Max)

Dummies features: 
  - ARTICULO: 20 new columns
  - CENTROCONSUMO: 2445 new columns
    --> CENTROCONSUMO has been removed
  - Mes: 12 new columns
Adding dummies to dataset...
Completed
Working dataset has: 67688 rows and 58 columns
Saving dataset to ./csv/clean_master_dummies_shorter.csv


# Modelo

## Short:

In [48]:
# Identfy the file location of the master dataset
I_data_path = './csv/clean_master_dummies_shorter.csv' 
I_Model_path = './3_4_2020/Short/128'

# Parameters
k = 4
num_epochs = 100
batch_Size = 128
Verbose = 0  #1 ó 0

# Types of Models (Possibilities: 1 to 6, you can choose how many you want e.g: [2,3,1,6])
NNTList = [1,2,3,4,5,6]  

# Function
train_data, train_targets, test_data, test_targets = NNTRegressor(I_data_path, I_Model_path, k, num_epochs, 
                                                                  batch_Size, Verbose, NNTList)

Loading the data...
Working dataset has: 67688 rows and 59 columns
Normalising...
save scaler to ./3_4_2020/Short/128temp/scaler.pickle
Completed
Spliting in train and test set
Saving training feature set to ./3_4_2020/Short/128temp/train_X.csv
Saving training labels to ./3_4_2020/Short/128temp/train_Y.csv
Saving test feature set to ./3_4_2020/Short/128temp/test_X.csv
Saving test labels to ./3_4_2020/Short/128/temp/test_Y.csv
Distributions of CANTIDADSALIDA
- Starting to train the model:
Model 1:
   Processing fold # 0


KeyError: 'mean_absolute_error'

In [ ]:
# Identfy the file location of the master dataset
I_data_path = './csv/clean_master_dummies_shorter.csv' 
I_Model_path = './3_5_2020/Short/16'

# Parameters
k = 4
num_epochs = 100
batch_Size = 16
Verbose = 0  #1 ó 0

# Types of Models (Possibilities: 1 to 6, you can choose how many you want e.g: [2,3,1,6])
NNTList = [1, 2, 3, 4, 5, 6]  

# Function
train_data, train_targets, test_data, test_targets = NNTRegressor(I_data_path, I_Model_path, k, num_epochs, 
                                                                  batch_Size, Verbose, NNTList)

## Medium:

In [ ]:
# Identfy the file location of the master dataset
I_data_path = './csv/clean_master_dummies_Medium.csv' 
I_Model_path = './3_4_2020/Medium/128'

# Parameters
k = 4
num_epochs = 100
batch_Size = 128
Verbose = 0  #1 ó 0

# Types of Models (Possibilities: 1 to 6, you can choose how many you want e.g: [2,3,1,6])
NNTList = [1,2,3,4,5,6]  

# Function
train_data, train_targets, test_data, test_targets = NNTRegressor(I_data_path, I_Model_path, k, num_epochs, 
                                                                  batch_Size, Verbose, NNTList)

## Big:

In [ ]:
# Identfy the file location of the master dataset
I_data_path = './csv/clean_master_dummies_Big.csv' 
I_Model_path = './3_4_2020/Big/128'

# Parameters
k = 4
num_epochs = 100
batch_Size = 128
Verbose = 0  #1 ó 0

# Types of Models (Possibilities: 1 to 6, you can choose how many you want e.g: [2,3,1,6])
NNTList = [1,2,3,4,5,6]  

# Function
train_data, train_targets, test_data, test_targets = NNTRegressor(I_data_path, I_Model_path, k, num_epochs, 
                                                                  batch_Size, Verbose, NNTList)